# Techniques for sampling parameter space - Halton Sampling

Halton sampling is a quasi-random sampling technique based on the halton sequence. Using a quasi-random number sequence based on coprime numbers, halton sampling allows entire parameter spaces to be sampled efficiently and with an element of random nature. Like random sampling, halton sampling allows data trends across the entire parameter space to be observed with a small number of samples, but distributes the sample points more efficiently throughout the parameter space.

The following code runs a neutronics simulation using a simple pre-defined model. The code varies the input parameters according to the halton sequence. Li6 enrichment and breeder percent in breeder plus multiplier ratio are both varied between 0 and 100.

In [ ]:
from pathlib import Path
import json
import uuid
import argparse

import ghalton
from tqdm import tqdm

from openmc_model import find_tbr_hcpb


number_of_simulations = 16


sequencer = ghalton.Halton(2)
coords = sequencer.get(number_of_simulations)

print("running simulations with halton sampling")

for coord in tqdm(coords):

    breeder_percent_in_breeder_plus_multiplier_ratio = coord[0] * 100  # scales sampling from 0 to 100
    blanket_breeder_li6_enrichment = coord[1] * 100  # scales sampling from 0 to 100

    result = find_tbr_hcpb(breeder_percent_in_breeder_plus_multiplier_ratio,
                           blanket_breeder_li6_enrichment)

    result["sample"] = "halton"

    filename = "outputs/" + str(uuid.uuid4()) + ".json"
    Path(filename).parent.mkdir(parents=True, exist_ok=True)
    with open(filename, mode="w", encoding="utf-8") as f:
        json.dump(result, f, indent=4)

print("results saved in outputs folder")

As you can see, the sample points have a much better spatial distribution across the parameter shapce than random sampling.

The main advantage of halton sampling is that it allows the efficient sampling of an entire parameter space. Sample points are not clustered meaning each simulation provides a large amount of additional information about the overall data trend, and its quasi-random nature means additional samples can be added efficiently. However, the main disadvantage of halton sampling is that it still surveys the entire parameter space of interest. This can lead to the excessive sampling of 'flat' regions of the parameter space, i.e. areas where there is little variation in the overall trend, resulting in inefficiency.

Overall, halton sampling is better than random and grid sampling as it provides good spatial distribution and allows more samples to be added easily and efficiently. Ideally, however, we want to avoid over-sampling flat regions of the parameter space to further improve efficiency and reduce the number of simulations required. This is where 'adaptive' sampling is advantageous.